# 推断
在本课程中，您将从产品评论和新闻文章中推断情感和主题。

[课程网站](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/5/inferring)

## 准备

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 产品的评论文本

In [4]:
lamp_review = """
需要一盏适合卧室的漂亮台灯，这款台灯还有额外的存储空间，价格不高。收到货很快。
运输途中灯的拉绳断了，公司很高兴地给我寄了一个新的。几天之内就收到了。
组装起来很容易。我遇到了一个缺失的零件，于是联系了他们的客服，他们很快就给我寄来了缺失的部件！
Lumina 对我来说是一个非常关心客户和产品的优秀公司！
"""

## 情感（积极/消极）

In [5]:
prompt = f"""
以下产品评论的情感是什么？评论文本由三个反引号分隔。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

积极的。


In [6]:
prompt = f"""
以下产品评论的情感是什么？评论文本由三个反引号分隔。

将您的答案以单个词汇给出，选择“正面”或“负面”。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

正面


## 识别愤怒

In [7]:
prompt = f"""
作者是否在以下评论中表达了愤怒？评论由三个反引号分隔。请回答“yes”或“no”。

评论内容：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

no


## 从客户评论中提取产品和公司名称

In [8]:
prompt = f"""
请从以下评论文本中找出以下信息：

评论者购买的物品
制造该物品的公司
评论由三个反引号分隔。请将您的回答格式化为一个 JSON 对象，其中 "Item" 和 "Brand" 作为键。如果信息不存在，请使用 "unknown" 作为值。请尽可能简短地回答。

评论内容：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "台灯",
  "Brand": "Lumina"
}


## 同时进行多项任务

In [9]:
prompt = f"""
请从以下评论文本中找出以下信息：
- 情感（积极或消极）
- 评论者是否表达愤怒？（true 或 false）
- 评论者购买的物品
- 制造该物品的公司

评论由三个反引号分隔。请将您的回答格式化为一个 JSON 对象，其中 "Sentiment"、"Anger"、"Item" 和 "Brand" 作为键。
如果信息不存在，请使用 "unknown" 作为值。请尽可能简短地回答。将 "Anger" 的值格式化为布尔值。

评论内容：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "积极",
  "Anger": false,
  "Item": "漂亮台灯",
  "Brand": "Lumina"
}


## 推断主题

In [10]:
story = """
在最近一次政府进行的调查中，公共部门员工被要求对他们所在部门的满意程度进行评价。结果显示，美国国家航空航天局（NASA）是最受欢迎的部门，满意度评分为95%。

一位 NASA 的员工，约翰·史密斯对这一调查结果发表了评论，他说：“我并不惊讶于 NASA 能够脱颖而出。这是一个和出色的人共事并拥有难以置信机会的好地方。我为成为这样一个创新组织的一员感到自豪。”

NASA 的管理团队也对这一结果表示欢迎，总监汤姆·约翰逊表示：“我们很高兴听到我们的员工对在 NASA 的工作表示满意。我们拥有一支才华横溢、敬业的团队，他们为实现我们的目标而不懈努力，看到他们的辛勤工作取得了回报真是太棒了。”

调查还显示，社会保障局的满意度评分最低，只有45%的员工表示对他们的工作感到满意。政府承诺将解决员工在调查中提出的问题，并努力提高所有部门的工作满意度。
"""

## 推断5个主题

In [11]:
prompt = f"""
确定以下文本中讨论的五个主题，文本由三个反引号标记分隔。

使每个项目长一个或两个单词。

将您的回答格式化为逗号分隔的项目列表。

文本示例： '''{story}'''
"""
response = get_completion(prompt)
print(response)

NASA, 员工满意度, 创新组织, 管理团队, 社会保障局


In [12]:
response.split(sep=',')

['NASA', ' 员工满意度', ' 创新组织', ' 管理团队', ' 社会保障局']

## 为特定主题制定新闻提醒

In [17]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [19]:
prompt = f"""
确定以下主题列表中的每个项目是否是下面文本中的主题，该文本由三个反引号分隔。

将您的答案以列表形式呈现，对于每个主题，用 0 或 1 表示。

主题列表：{", ".join(topic_list)}

文本示例：'''{story}'''
"""
response = get_completion(prompt)
print(response)

- nasa: 1
- local government: 0
- engineering: 0
- employee satisfaction: 1
- federal government: 1


In [20]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['- nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!
